# Sensor Position Optimization

### BMW Quantum Computing Challenge

#### Valter Uotila, Unified Database Management Systems, Department of Computer Science at University of Helsinki
#### Sardana Ivanova, Discovery Research Group, Department of Computer Science at University of Helsinki

## Remarks about the implementation

We have been learning, testing and developing some basic quantum algorithms using D-waves own API but we are not very experienced with details related to these quantum computers.

Unfortunately, it seems that Amazon Bracket does not yet support D-waves hybrid solvers. We might be wrong but we could not figure out how to access, for example, LeapHybridSampler using Bracket. It seems that BracketDWaveSampler and BracketSampler are using the standard DWaveSampler (https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html). Because our solution is based on LeapHybridSampler, it is not possible to run it in Bracket. Maybe in the future Amazon will implement something like BracketDWaveHybridSampler which offers the functionality of LeapHybridSampler. Anyway, we are positively impressed how Bracket collects various quantum computing resources together.

## Initializing parameters

### Importing D-wave packages

Note that in order to run the codes here, you need to be able to successfully access your D-wave quantum cloud computing resources. You can see more info at https://cloud.dwavesys.com/. We are used to develop with D-waves developer's plan which includes 1 min of quantum computing time per month.

In [1]:
import dimod

from dwave.system import LeapHybridSampler

import json
import itertools
import os
import math

from ipynb.fs.defs.sensor_covers_2D import sensor_covers

notebook_path = os.path.abspath("main_2D_connecting_Dwave_Leap.ipynb")

### Initializing sensors

In [2]:
abs_sensors_file_path = os.path.join(os.path.dirname(notebook_path), "2d_example_data/sensors2.json")

f = open(abs_sensors_file_path)

sensor_root = json.load(f)
sensors = sensor_root["sensors"]

# print(sensors)

### Initializing car

In [3]:
abs_car_file_path = os.path.join(os.path.dirname(notebook_path), "2d_example_data/car.json")

f = open(abs_car_file_path)

car = json.load(f)

car_grid = [] #[(0,3), (1,3), (2,3), (0,2), (1,2), (2,2), (0,1), (1,1), (2,1), (0,0), (1,0), (2,0)]

x_dim = [car["dimensions"]["v1"]["x"], car["dimensions"]["v2"]["x"], car["dimensions"]["v3"]["x"], car["dimensions"]["v4"]["x"]]
y_dim = [car["dimensions"]["v1"]["y"], car["dimensions"]["v2"]["y"], car["dimensions"]["v3"]["y"], car["dimensions"]["v4"]["y"]]
start_x = min(x_dim)
end_x = max(x_dim)
start_y = min(y_dim)
end_y = max(y_dim)

for x in range(start_x, end_x + 1):
        for y in range(start_y, end_y + 1):
            car_grid.append((x,y))

# print(car_grid)

### Initializing environment

In [4]:
environment = {"v1": (-8,8), "v2": (8,8), "v3": (8, -8), "v4": (-8,-8)}

def critical_value(x, y):
    return 1

## Constructing quadratic binary model

In [5]:
vartype = dimod.BINARY
main_bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, vartype)

# Initializing variables
# Variables are triples (x, y, i) where x and y are coordinate points in the space
# and i is the index of the sensor.

def create_variables(start_x, end_x, start_y, end_y, sensors):
    variables = []
    for x in range(start_x, end_x + 1):
        for y in range(start_y, end_y + 1):
            if (x,y) not in car_grid:
                variables.append((x, y))
    return variables

env = create_variables(environment["v1"][0], environment["v2"][0], environment["v3"][1], environment["v2"][1], sensors)

# print(len(env))

# Next we collect the variables which are assigned to the car surface

car_surface = []

for pos in car["positions"]:
    if pos["x1"] == pos["x2"]:
        x = pos["x1"]
        for y in range(pos["y1"], pos["y2"] + 1):
            car_surface.append((x, y))
    elif pos["y1"] == pos["y2"]:
        y = pos["y1"]
        for x in range(pos["x1"], pos["x2"] + 1):
            car_surface.append((x, y))
    else:
        raise ValueError('The allowed position on the car surface is not a line.')
        
# print(car_surface)

The following functions enable us to append or initialize coefficients for the variables in the BQM. The distance function implements the ordinary Euclidean distance.

In [6]:
def append_linear_safe(variable, value, linear_dict):
    if variable in linear_dict.keys():
        linear_dict[variable] = linear_dict[variable] + value
    else:
        linear_dict[variable] = value

def append_quadratic_safe(variable, value, quadratic_dict):
    if variable in quadratic_dict.keys():
        quadratic_dict[variable] = quadratic_dict[variable] + value
    else:
        quadratic_dict[variable] = value
        
def distance(x, y):
    return math.floor(math.sqrt(pow((x[0]- y[0]), 2) + pow((x[0]- y[0]), 2)))

### Constraint 1: variables go in pairs

Every binary quadratic function which is part of the model contains four parameters: linear terms, quadratic terms, offset (constant) and variable type. Variable type is always BINARY since we are using QUBO. If we would use Ising, we set variable type to be SPIN.

In [7]:
# Encoding constraint H1

A1 = 345*2
linear_h1 = {}
quadratic_h1 = {}
offset_h1 = 0.0

for sensor in sensors:
    for y in env:
        
        var_y = (y[0], y[1], sensor["id"])
        append_linear_safe(var_y, 1, linear_h1)
        
        for x in car_surface:
            
            var_x = (x[0], x[1], sensor["id"])
            
            append_linear_safe(var_x, 1, linear_h1)
            append_quadratic_safe((var_x, var_y), -2, quadratic_h1)
            
        for tupl in itertools.combinations(car_surface, 2):
            
            var_1 = (tupl[0][0], tupl[0][1], sensor["id"])
            var_2 = (tupl[1][0], tupl[1][1], sensor["id"])
            
            append_quadratic_safe((var_1, var_2), 2, quadratic_h1)
            
bqm_h1 = dimod.BinaryQuadraticModel(linear_h1, quadratic_h1, offset_h1, vartype)
bqm_h1.scale(A1)
main_bqm.update(bqm_h1)

            
# Encoding constraint H2

A2 = A1
linear_h2 = {}
quadratic_h2 = {}
offset_h2 = 0.0

for sensor in sensors:
    for x in car_surface:
        
        var_x = (x[0], x[1], sensor["id"])
        append_linear_safe(var_x, 1, linear_h2)
        
        for y in env:
            var_y = (y[0], y[1], sensor["id"])
            
            append_linear_safe(var_y, 1, linear_h2)
            
            append_quadratic_safe((var_x, var_y), -2, quadratic_h2)
            
        for tupl in itertools.combinations(env, 2):
            var_1 = (tupl[0][0], tupl[0][1], sensor["id"])
            var_2 = (tupl[1][0], tupl[1][1], sensor["id"])
            
            append_quadratic_safe((var_1, var_2), 2, quadratic_h2)
            
bqm_h2 = dimod.BinaryQuadraticModel(linear_h2, quadratic_h2, offset_h2, vartype)
bqm_h2.scale(A2)
main_bqm.update(bqm_h2)
            
#for elem in main_bqm.linear:
#        print(elem, main_bqm.linear[elem])

#for elem in main_bqm.quadratic:
#    print(elem, main_bqm.quadratic[elem])
            

### Constraint 2: range of sensor views

In [8]:
# Encoding constraint H3

A3 = A2
linear_h3 = {}
quadratic_h3 = {}
offset_h3 = 1

for sensor in sensors:
    for x in car_surface:
        for y in env:
            constant = float(pow(distance(x, y) - sensor["view"]["range"], 2)) + 1
            
            # print(constant)
            #if constant != 1:
            #    constant = pow(distance(x, y) + 1 - sensor["view"]["range"] + 1, 2)
            
            var_x = (x[0], x[1], sensor["id"])
            var_y = (y[0], y[1], sensor["id"])
            
            #append_linear_safe(var_x, pow(constant, 2), linear_h3)
            #append_linear_safe(var_y, pow(constant, 2), linear_h3)
            
            append_quadratic_safe((var_x, var_y), float(pow(constant, 2)) - 2*constant, quadratic_h3)
            
bqm_h3 = dimod.BinaryQuadraticModel(linear_h3, quadratic_h3, offset_h3, vartype)
bqm_h3.scale(A3)
main_bqm.update(bqm_h3)
    

### Constraint 3: maximizing sensor view coverage

Other constaints try to minimize the number of sensors. Thus the trivial solution to these is not to include any sensors. On the other hand, this part of the model tries to maximize the coverage. This creates a sort of tension between different parts of this optimization problem which eventually unstabilizes the energy and creates some non-trivial solution to the problem.

In [9]:
# Encoding constraint H4

total_points = len(env)

A4 = 2*2 # Seems to have huge affect on the result
linear_h4 = {}
quadratic_h4 = {}
offset_h4 = float(pow(total_points, 2))

for sensor in sensors:
    for x in car_surface:
        for y in env:
            covered_area = float(pow(sensor["view"]["range"], 2))*float(math.tan(math.pi * float(sensor["view"]["angle"] / 360)))
            
            quadratic_constant = float(pow(covered_area, 2)) - 2*covered_area*total_points
            
            var_x = (x[0], x[1], sensor["id"])
            var_y = (y[0], y[1], sensor["id"])
            
            append_quadratic_safe((var_x, var_y), quadratic_constant, quadratic_h4)
            
bqm_h4 = dimod.BinaryQuadraticModel(linear_h4, quadratic_h4, offset_h4, vartype)
bqm_h4.scale(A4)
main_bqm.update(bqm_h4)
            

### Constraint 4: optimizing overlap of sensor views

In [10]:
# Encoding constraint H5

for p in env:
    cr = critical_value(p[0], p[1])
    A5 = 40
    linear_h5 = {}
    quadratic_h5 = {}
    offset_h5 = float(pow(cr, 2))
    
    for sensor in sensors:
        for x in car_surface:
            for y in env:
                covers = sensor_covers(x, y, p, sensor)
                
                var_x = (x[0], x[1], sensor["id"])
                var_y = (y[0], y[1], sensor["id"])
            
                append_quadratic_safe((var_x, var_y), float(pow(covers, 2) - 2*cr*covers), quadratic_h5)
                
    bqm_h5 = dimod.BinaryQuadraticModel(linear_h5, quadratic_h5, offset_h5, vartype)
    bqm_h5.scale(A5)
    main_bqm.update(bqm_h5)

### Constraint 5: minimizing total price

In [11]:
# Encoding constraint H6

A6 = 1000
linear_h6 = {}
quadratic_h6 = {}
offset_h6 = 0.0

for sensor in sensors:
    for x in car_surface:
        for y in env:
            var_x = (x[0], x[1], sensor["id"])
            var_y = (y[0], y[1], sensor["id"])
            
            append_quadratic_safe((var_x, var_y), sensor["price"], quadratic_h6)

bqm_h6 = dimod.BinaryQuadraticModel(linear_h6, quadratic_h6, offset_h6, vartype)
bqm_h6.scale(A6)
main_bqm.update(bqm_h6)

#for elem in main_bqm.linear:
#        print(elem, main_bqm.linear[elem])

#for elem in main_bqm.quadratic:
#    if main_bqm.quadratic[elem] != 16:
#        print(elem, main_bqm.quadratic[elem])

### Solve QUBO

Unfortunately, LeapHybridSampler is not available in Amazon Bracket. That is why this code will not work in Bracket. On the other hand, we tried run the code using BracketDwaveSampler but the problem cannot be mapped to the circuits.

In [12]:
main_bqm.normalize()

sampler = LeapHybridSampler()
sampleset = sampler.sample(main_bqm, label='BMW Sensor Position', time_limit = 3)
sample = sampleset.first.sample
print(sampleset)
print()
# energy = sampleset.first.energy
print("Possible sensor positions in the space (x-coordinate, y-coordinate, sensor id):")
i = 0
for varname, value in sample.items():
    if value == 1:
        i+=1
        print(varname, value)
print(i)

  (-8, -8, 1) (-8, -8, 2) (-8, -7, 1) ... (8, 8, 2)    energy num_oc.
0           0           0           0 ...         0 -0.102623       1
['BINARY', 1 rows, 1 samples, 570 variables]

Possible sensor positions in the space (x-coordinate, y-coordinate, sensor id):
(-6, -8, 1) 1
(-6, -7, 1) 1
(-6, -6, 1) 1
(-6, -5, 1) 1
(-6, -4, 1) 1
(-6, -3, 1) 1
(-6, -2, 1) 1
(-6, -1, 1) 1
(-6, 1, 1) 1
(-6, 5, 1) 1
(-6, 7, 1) 1
(-6, 8, 1) 1
(-5, -8, 1) 1
(-5, -7, 1) 1
(-5, -6, 1) 1
(-5, -5, 1) 1
(-5, -4, 1) 1
(-5, -3, 1) 1
(-5, -2, 1) 1
(-5, -1, 1) 1
(-5, 1, 1) 1
(-5, 2, 1) 1
(-5, 3, 1) 1
(-5, 4, 1) 1
(-5, 5, 1) 1
(-5, 6, 1) 1
(-5, 7, 1) 1
(-5, 8, 1) 1
(0, 0, 1) 1
(0, 1, 1) 1
(0, 2, 1) 1
(0, 3, 1) 1
(2, 0, 1) 1
(2, 1, 1) 1
(2, 2, 1) 1
(2, 3, 1) 1
(7, -8, 1) 1
(7, -7, 1) 1
(7, -6, 1) 1
(7, -5, 1) 1
(7, -4, 1) 1
(7, -3, 1) 1
(7, -2, 1) 1
(7, 4, 1) 1
(7, 5, 1) 1
(7, 6, 1) 1
(7, 7, 1) 1
(7, 8, 1) 1
(8, -8, 1) 1
(8, -7, 1) 1
(8, -6, 1) 1
(8, -5, 1) 1
(8, -4, 1) 1
(8, -3, 1) 1
(8, 0, 1) 1
(8, 8, 1) 1
56
